In [ ]:
from pathlib import Path

seeds = range(1, 11)

# Config should match Problem init

base_path = Path("smacbenchmarking/configs/problem")

# Problem ids
benchmark_id_bbob = "BBOB"

# BBOB

In [ ]:
# BBOB
from omegaconf import DictConfig, OmegaConf

import shutil

target_path = base_path / benchmark_id_bbob
if target_path.is_dir():
    shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

problem_class = "smacbenchmarking.benchmarks.bbob.BBOBProblem"

fids = list(range(1, 25))
instances = [0]
dimensions = [4]
n_trials = {
    4: 100,
}

for fid in fids:
    for inst in instances:
        for dim in dimensions:
            problem_id = f"noiseless/{dim}/{fid}/{inst}"
            cfg = DictConfig({
                "benchmark_id": benchmark_id_bbob,
                "problem_id": problem_id,
                "problem": {
                    # "problem_class": {
                        "_target_": problem_class,
                        "dimension": dim,
                        "fid": fid,
                        "instance": inst,
                        "seed": "${seed}",  # get the seed from global cfg                      
                    # },
                },
                "task": {
                    "n_trials": n_trials[dim],
                    "n_objectives": 1,
                }
            
            })
            fn = target_path / f"cfg_{dim}_{fid}_{dim}_{inst}.yaml"
            yaml_str = OmegaConf.to_yaml(cfg=cfg)
            yaml_str = "# @package _global_\n" + yaml_str
            fn.write_text(yaml_str)
            print(cfg)


# HPOB

In [ ]:
# HPOB
from operator import attrgetter
import re
from pathlib import Path

base_path = Path("smacbenchmarking/configs/problem")

# Problem ids
benchmark_id = "HPOB"

surrogates_dir = Path('smacbenchmarking/benchmark_data/HPO-B/saved-surrogates')

all_res = map(attrgetter('name'), surrogates_dir.glob('surrogate-*'))

from omegaconf import DictConfig, OmegaConf
import shutil

target_path = base_path / benchmark_id
if target_path.is_dir():
    shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

problem_class = "smacbenchmarking.benchmarks.hpo_b.HPOBProblem"
n_trials = 100

for res in all_res:
    model_id, dataset_id = re.findall('\d+', res)
    problem_id = f'hpob/{model_id}/{dataset_id}'
    cfg = DictConfig({
        "benchmark_id": benchmark_id,
        "problem_id": problem_id,
        "problem": {
            # "problem_class": {
            "_target_": problem_class,
            "model_id": model_id,
            "dataset_id": dataset_id,
            "surrogates_dir": str(surrogates_dir)
            # },
        },
        "task": {
            "n_trials": n_trials,
            "n_objectives": 1,
        }

    })
    fn = target_path / f"cfg_{model_id}_{dataset_id}.yaml"
    yaml_str = OmegaConf.to_yaml(cfg=cfg)
    yaml_str = "# @package _global_\n" + yaml_str
    fn.write_text(yaml_str)

# YAHPO

In [ ]:
COMBIS = {'lcbench': ['3945', '7593', '34539', '126025', '126026', '126029', '146212', '167104',
                      '167149', '167152', '167161', '167168', '167181', '167184', '167185',
                      '167190', '167200', '167201', '168329', '168330', '168331', '168335',
                      '168868', '168908', '168910', '189354', '189862', '189865', '189866',
                      '189873', '189905', '189906', '189908', '189909'],
          'fcnet': ['fcnet_naval_propulsion', 'fcnet_protein_structure',
                    'fcnet_slice_localization', 'fcnet_parkinsons_telemonitoring'],
          'nb301': ['CIFAR10'],
          'rbv2_svm': ['40981', '4134', '1220', '40978', '40966', '40536', '41156', '458',
                       '41157', '40975', '40994', '1468', '6332', '40670', '151', '1475',
                       '1476', '1478', '1479', '41212', '1480', '1053', '1067', '1056', '12',
                       '1487', '1068', '32', '470', '312', '38', '40982', '50', '41216', '307',
                       '40498', '181', '1464', '41164', '16', '1461', '41162', '6', '14',
                       '1494', '54', '375', '1590', '23', '41163', '1111', '41027', '40668',
                       '41138', '4135', '4538', '40496', '4534', '40900', '1457', '11', '1462',
                       '41142', '40701', '29', '37', '23381', '188', '41143', '1063', '3', '18',
                       '40979', '22', '1515', '334', '24', '1493', '28', '1050', '1049',
                       '40984', '40685', '42', '44', '46', '1040', '41146', '377', '40499',
                       '1497', '60', '40983', '4154', '469', '31', '41278', '1489', '1501',
                       '15', '300', '1485', '1486', '1510', '182', '41169'],
          'rbv2_ranger': ['4135', '40981', '4134', '1220', '4154', '4538', '40978', '375',
                          '40496', '40966', '4534', '40900', '40536', '41156', '1590', '1457',
                          '458', '469', '41157', '11', '1461', '1462', '1464', '15', '40975',
                          '41142', '40701', '40994', '23', '1468', '40668', '29', '31', '6332',
                          '37', '40670', '23381', '151', '188', '41164', '1475', '1476', '1478',
                          '1479', '41212', '1480', '41143', '1053', '41027', '1067', '1063',
                          '3', '6', '1485', '1056', '12', '14', '16', '18', '40979', '22',
                          '1515', '334', '24', '1486', '41278', '28', '1487', '1068', '1050',
                          '1049', '32', '1489', '470', '1494', '182', '312', '40984', '1501',
                          '40685', '38', '42', '44', '46', '40982', '1040', '41146', '377',
                          '40499', '50', '54', '41216', '307', '1497', '60', '1510', '40983',
                          '40498', '181', '41138', '41163', '1111', '41159', '300', '41162',
                          '23517', '41165', '4541', '41161', '41166', '40927', '41150', '23512',
                          '41168', '1493', '40996', '554', '40923', '41169'],
          'rbv2_rpart': ['41138', '4135', '40981', '4134', '40927', '1220', '4154', '40923',
                         '41163', '40996', '4538', '40978', '375', '1111', '40496', '40966',
                         '41150', '4534', '40900', '40536', '41156', '1590', '1457', '458',
                         '469', '41157', '11', '1461', '1462', '1464', '15', '40975', '41142',
                         '40701', '40994', '23', '1468', '40668', '29', '31', '6332', '37',
                         '4541', '40670', '23381', '151', '188', '41164', '1475', '1476',
                         '41159', '1478', '41169', '23512', '1479', '41212', '1480', '300',
                         '41168', '41143', '1053', '41027', '1067', '1063', '41162', '3', '6',
                         '1485', '1056', '12', '14', '16', '18', '40979', '22', '1515', '554',
                         '334', '24', '1486', '23517', '1493', '28', '1487', '1068', '1050',
                         '1049', '32', '1489', '470', '1494', '41161', '41165', '182', '312',
                         '40984', '1501', '40685', '38', '42', '44', '46', '40982', '1040',
                         '41146', '377', '40499', '50', '54', '41166', '307', '1497', '60',
                         '1510', '40983', '40498', '181'],
          'rbv2_glmnet': ['41138', '4135', '40981', '4134', '1220', '4154', '41163', '4538',
                          '40978', '375', '1111', '40496', '40966', '41150', '4534', '40900',
                          '40536', '41156', '1590', '1457', '458', '469', '41157', '11', '1461',
                          '1462', '1464', '15', '40975', '41142', '40701', '40994', '23',
                          '1468', '40668', '29', '31', '6332', '37', '4541', '40670', '23381',
                          '151', '188', '41164', '1475', '1476', '41159', '1478', '41169',
                          '23512', '1479', '41212', '1480', '300', '41168', '41143', '1053',
                          '41027', '1067', '1063', '41162', '3', '6', '1485', '1056', '12',
                          '14', '16', '18', '40979', '22', '1515', '334', '24', '1486', '23517',
                          '41278', '1493', '28', '1487', '1068', '1050', '1049', '32', '1489',
                          '470', '1494', '41161', '182', '312', '40984', '1501', '40685', '38',
                          '42', '44', '46', '40982', '1040', '41146', '377', '40499', '50',
                          '54', '41216', '41166', '307', '1497', '60', '1510', '40983', '40498',
                          '181', '554'],
          'rbv2_xgboost': ['16', '40923', '41143', '470', '1487', '40499', '40966', '41164',
                           '1497', '40975', '1461', '41278', '11', '54', '300', '40984', '31',
                           '1067', '1590', '40983', '41163', '41165', '182', '1220', '41159',
                           '41169', '42', '188', '1457', '1480', '6332', '181', '1479', '40670',
                           '40536', '41138', '41166', '6', '14', '29', '458', '1056', '1462',
                           '1494', '40701', '12', '1493', '44', '307', '334', '40982', '41142',
                           '38', '1050', '469', '23381', '41157', '15', '4541', '23', '4134',
                           '40927', '40981', '41156', '3', '1049', '40900', '1063', '23512',
                           '40979', '1040', '1068', '41161', '22', '1489', '41027', '24',
                           '4135', '23517', '1053', '1468', '312', '377', '1515', '18', '1476',
                           '1510', '41162', '28', '375', '1464', '40685', '40996', '41146',
                           '41216', '40668', '41212', '32', '60', '4538', '40496', '41150',
                           '37', '46', '554', '1475', '1485', '1501', '1111', '4534', '41168',
                           '151', '4154', '40978', '40994', '50', '1478', '1486', '40498'],
          'rbv2_aknn': ['41138', '40981', '4134', '40927', '1220', '4154', '41163', '40996',
                        '4538', '40978', '375', '1111', '40496', '40966', '41150', '4534',
                        '40900', '40536', '41156', '1590', '1457', '458', '469', '41157', '11',
                        '1461', '1462', '1464', '15', '40975', '41142', '40701', '40994', '23',
                        '1468', '40668', '29', '31', '6332', '37', '4541', '40670', '23381',
                        '151', '188', '41164', '1475', '1476', '41159', '1478', '41169',
                        '23512', '1479', '41212', '1480', '300', '41168', '41143', '1053',
                        '41027', '1067', '1063', '41162', '3', '6', '1485', '1056', '12', '14',
                        '16', '18', '40979', '22', '1515', '554', '334', '24', '1486', '23517',
                        '41278', '1493', '28', '1487', '1068', '1050', '1049', '32', '1489',
                        '470', '1494', '41161', '41165', '182', '312', '40984', '1501', '40685',
                        '38', '42', '44', '46', '40982', '1040', '41146', '377', '40499', '50',
                        '54', '41216', '41166', '307', '1497', '60', '1510', '40983', '40498',
                        '181', '40923'],
          'rbv2_super': ['41138', '40981', '4134', '1220', '4154', '41163', '4538', '40978',
                         '375', '1111', '40496', '40966', '4534', '40900', '40536', '41156',
                         '1590', '1457', '458', '469', '41157', '11', '1461', '1462', '1464',
                         '15', '40975', '41142', '40701', '40994', '23', '1468', '40668', '29',
                         '31', '6332', '37', '40670', '23381', '151', '188', '41164', '1475',
                         '1476', '1478', '41169', '1479', '41212', '1480', '300', '41143',
                         '1053', '41027', '1067', '1063', '41162', '3', '6', '1485', '1056',
                         '12', '14', '16', '18', '40979', '22', '1515', '334', '24', '1486',
                         '1493', '28', '1487', '1068', '1050', '1049', '32', '1489', '470',
                         '1494', '182', '312', '40984', '1501', '40685', '38', '42', '44', '46',
                         '40982', '1040', '41146', '377', '40499', '50', '54', '307', '1497',
                         '60', '1510', '40983', '40498', '181'],
          'iaml_ranger': ['40981', '41146', '1489', '1067'],
          'iaml_rpart': ['40981', '41146', '1489', '1067'],
          'iaml_glmnet': ['40981', '41146', '1489', '1067'],
          'iaml_xgboost': ['40981', '41146', '1489', '1067'],
          'iaml_super': ['40981', '41146', '1489', '1067']}

UPPER = {'trainsize': 1.0,
         'repl': 10,
         'epochs': {
             'lcbench': 52,
             'fcnet': 100,
             'nb301': 98,
         }}
LOWER = {'trainsize': 0.03,
         'repl': 1,
         'epochs': {
             'lcbench': 1,
             'fcnet': 1,
             'nb301': 1,
         }}

FIDELITY_TYPES = {'lcbench': ['epoch'],
                  'fcnet': ['epoch'],
                  'nb301': ['epoch'],
                  'rbv2_svm': ['trainsize', 'repl'],
                  'rbv2_ranger': ['trainsize', 'repl'],
                  'rbv2_rpart': ['trainsize', 'repl'],
                  'rbv2_glmnet': ['trainsize', 'repl'],
                  'rbv2_xgboost': ['trainsize', 'repl'],
                  'rbv2_aknn': ['trainsize', 'repl'],
                  'rbv2_super': ['trainsize', 'repl'],
                  'iaml_ranger': ['trainsize'],
                  'iaml_rpart': ['trainsize'],
                  'iaml_glmnet': ['trainsize'],
                  'iaml_xgboost': ['trainsize'],
                  'iaml_super': ['trainsize']}

METRICS = {'lcbench': ['time', 'val_accuracy', 'val_cross_entropy', 'val_balanced_accuracy',
                       'test_cross_entropy', 'test_balanced_accuracy'],
           'fcnet': ['valid_loss', 'valid_mse', 'runtime', 'n_params'],
           'nb301': ['val_accuracy', 'runtime'],
           'rbv2_svm': ['acc', 'bac', 'auc', 'brier', 'f1', 'logloss', 'timetrain', 'timepredict',
                        'memory'],
           'rbv2_ranger': ['acc', 'bac', 'auc', 'brier', 'f1', 'logloss', 'timetrain',
                           'timepredict', 'memory'],
           'rbv2_rpart': ['acc', 'bac', 'auc', 'brier', 'f1', 'logloss', 'timetrain', 'timepredict',
                          'memory'],
           'rbv2_glmnet': ['acc', 'bac', 'auc', 'brier', 'f1', 'logloss', 'timetrain',
                           'timepredict', 'memory'],
           'rbv2_xgboost': ['acc', 'bac', 'auc', 'brier', 'f1', 'logloss', 'timetrain',
                            'timepredict', 'memory'],
           'rbv2_aknn': ['acc', 'bac', 'auc', 'brier', 'f1', 'logloss', 'timetrain', 'timepredict',
                         'memory'],
           'rbv2_super': ['acc', 'bac', 'auc', 'brier', 'f1', 'logloss', 'timetrain', 'timepredict',
                          'memory'],
           'iaml_ranger': ['mmce', 'f1', 'auc', 'logloss', 'ramtrain', 'rammodel', 'rampredict',
                           'timetrain', 'timepredict', 'mec', 'ias', 'nf'],
           'iaml_rpart': ['mmce', 'f1', 'auc', 'logloss', 'ramtrain', 'rammodel', 'rampredict',
                          'timetrain', 'timepredict', 'mec', 'ias', 'nf'],
           'iaml_glmnet': ['mmce', 'f1', 'auc', 'logloss', 'ramtrain', 'rammodel', 'rampredict',
                           'timetrain', 'timepredict', 'mec', 'ias', 'nf'],
           'iaml_xgboost': ['mmce', 'f1', 'auc', 'logloss', 'ramtrain', 'rammodel', 'rampredict',
                            'timetrain', 'timepredict', 'mec', 'ias', 'nf'],
           'iaml_super': ['mmce', 'f1', 'auc', 'logloss', 'ramtrain', 'rammodel', 'rampredict',
                          'timetrain', 'timepredict', 'mec', 'ias', 'nf']}

# select your prefered target variable
iaml = 'f1'
rbv2 = 'acc'
TARGETMETRIC = {'lcbench': 'val_accuracy',
                'fcnet': 'valid_mse',
                'nb301': 'val_accuracy',
                'rbv2_svm': rbv2,
                'rbv2_ranger': rbv2,
                'rbv2_rpart': rbv2,
                'rbv2_glmnet': rbv2,
                'rbv2_xgboost': rbv2,
                'rbv2_aknn': rbv2,
                'rbv2_super': rbv2,
                'iaml_ranger': iaml,
                'iaml_rpart': iaml,
                'iaml_glmnet': iaml,
                'iaml_xgboost': iaml,
                'iaml_super': iaml}

LOWER_IS_BETTER = {
    'lcbench': False,
    'fcnet': True,
    'nb301': False,
    'rbv2_svm': False,
    'rbv2_ranger': False,
    'rbv2_rpart': False,
    'rbv2_glmnet': False,
    'rbv2_xgboost': False,
    'rbv2_aknn': False,
    'rbv2_super': False,
    'iaml_ranger': False,
    'iaml_rpart': False,
    'iaml_glmnet': False,
    'iaml_xgboost': False,
    'iaml_super': False,

}

In [ ]:
target_path = base_path / 'YAHPO'

if target_path.is_dir():
    shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

problem_class = "smacbenchmarking.benchmarks.yahpo.YahpoProblem"

fids = list(range(1, 25))
instances = [0]
dimensions = [4]
n_trials = {
    4: 100,
}

for bench, instance_list in COMBIS.items():
    for instance in instance_list:
        for fid_type in FIDELITY_TYPES[bench]:

            if fid_type == 'epoch':
                lower = LOWER['epochs'][bench]
                upper = UPPER['epochs'][bench]

            else:
                lower = LOWER[fid_type]
                upper = UPPER[fid_type]

            problem_id = f"yahpo/{bench}/{instance}/{fid_type}"

            cfg = DictConfig({
                'benchmark_id': 'YAHPO',
                'problem_id': problem_id,
                'problem': {
                    '_target_': problem_class,
                    'bench': bench,
                    'instance': instance,
                    'budget_type': fid_type,
                    'lower_is_better': LOWER_IS_BETTER[bench],
                    'metric': TARGETMETRIC[bench],
                },
                'task': {
                    'n_trials': 100,
                    'n_objectives': 1,
                    'min_budget': lower,
                    'max_budget': upper,
                }
            })


            fn = target_path / f"cfg_{bench}_{instance}_{fid_type}.yaml"
            yaml_str = OmegaConf.to_yaml(cfg=cfg)
            yaml_str = "# @package _global_\n" + yaml_str
            fn.write_text(yaml_str)
            print(cfg)